In [13]:
import sys
import os

parent = os.path.dirname(os.getcwd())
sys.path.insert(0, parent)

import numpy as np
import matplotlib.pyplot as plt

from Helpers import loadData, multiplot
from skimage.feature import canny
from scipy.ndimage import maximum_filter

In [3]:
data = loadData(location=r'/home/snow/Coding/Granular-Compression/Data',
                fileName='processedDown_17.hdf5')

In [ ]:
def CircleOverlap(c1, c2):
    x1, y1, r1 = c1
    x2, y2, r2 = c2
    d = hypot(x1 - x2, y1 - y2)

    if d >= r1 + r2:  # No overlap
        return 0.0
    if d <= abs(r1 - r2):  # One inside the other
        return (pi * min(r1, r2)**2) / (pi * max(r1, r2)**2)

    r1_sq = r1**2
    r2_sq = r2**2

    alpha = acos((d**2 + r1_sq - r2_sq) / (2 * d * r1))
    beta = acos((d**2 + r2_sq - r1_sq) / (2 * d * r2))

    area1 = r1_sq * alpha
    area2 = r2_sq * beta
    area3 = 0.5 * sqrt((-d + r1 + r2) * (d + r1 - r2) * (d - r1 + r2) * (d + r1 + r2))

    intersection = area1 + area2 - area3
    union = pi * r1_sq + pi * r2_sq - intersection
    return intersection / union

In [5]:
def CircleDetection(img, min_r=10, max_r=60, step=2,
                    sigma=2.75,threshold=100,iou_thresh=0.25):
    
    # Edge detection
    edges = canny(img, sigma=sigma)
    h, w = edges.shape

    # Radii
    radii = np.arange(min_r, max_r, step)
    num_radii = len(radii)

    # Precompute circle perimeters (stacked)
    theta = np.arange(0, 2 * pi, pi / 180)
    cos_t, sin_t = np.cos(theta), np.sin(theta)
    dx_all = np.round(radii[:, None] * cos_t).astype(int)
    dy_all = np.round(radii[:, None] * sin_t).astype(int)

    # Voting
    y_idxs, x_idxs = np.nonzero(edges)  # edge points (N,)
    N = len(x_idxs)

    # Expand edge coords against circle offsets
    # Shape: (num_radii, N, num_theta)
    x_c = x_idxs[None, :, None] - dx_all[:, None, :]
    y_c = y_idxs[None, :, None] - dy_all[:, None, :]

    # Keep only valid centers
    valid = (x_c >= 0) & (x_c < w) & (y_c >= 0) & (y_c < h)

    # Flatten into 1D indices for bincount
    flat_idx = (y_c * w + x_c) * num_radii + np.arange(num_radii)[:, None, None]
    flat_idx = np.where(valid, flat_idx, -1).ravel()

    flat_idx = flat_idx[flat_idx >= 0]  # remove invalid

    accumulator = np.bincount(flat_idx, minlength=h * w * num_radii).reshape(h, w, num_radii)

    # Candidate detection
    candidates = []
    for r_index, r in enumerate(radii):
        acc_slice = accumulator[:, :, r_index]
        local_max = (maximum_filter(acc_slice, size=5) == acc_slice)
        mask = (acc_slice > threshold) & local_max
        coords = np.argwhere(mask)

        if coords.size > 0:
            vals = acc_slice[coords[:, 0], coords[:, 1]]
            for (y, x), val in zip(coords, vals):
                neighbors = []
                for offset in [-2, -1, 1, 2]:
                    neighbor_idx = r_index + offset
                    if 0 <= neighbor_idx < num_radii:
                        neighbors.append(accumulator[y, x, neighbor_idx])
                if all(val > n for n in neighbors):
                    candidates.append((x, y, r, val))

    # IoU filtering
    final_circles = []
    for x, y, r, val in sorted(candidates, key=lambda c: -c[3]):
        this_circle = (x, y, r)
        keep = True
        for fx, fy, fr in final_circles:
            if CircleOverlap(this_circle, (fx, fy, fr)) > iou_thresh:
                keep = False
                break
        if keep:
            final_circles.append(this_circle)

    return final_circles

In [22]:
np.bincount([1,5,6,3,8,1])

array([0, 2, 0, 1, 0, 1, 1, 0, 1])

In [47]:
d = np.arange(radCount)[:,None,None]
m = np.reshape(np.arange(radCount),shape=(15,1,1))
print(radii.shape)
print(radii[:,None].shape)
print(cosT.shape)
print(rCent.shape)
print(r,c)
print(flatIdx)


(15,)
(15, 1)
(360,)
(15, 948, 360)
412 512
[     0      0      0 ... 210914 203234 195554]


In [49]:
minR,maxR = [30,60]
sigma = 2.75
img = data[:,:,20]

edgeMap = canny(img, sigma=2.75)
r,c = img.shape

radii = np.arange(minR,maxR,step=2)
radCount = len(radii)

theta = np.arange(0, 2*np.pi, np.pi/180)
cosT,sinT=np.cos(theta),np.sin(theta)
rAll = np.round(radii[:,None]*cosT).astype(int)
cAll = np.round(radii[:,None]*sinT).astype(int)

rIdx,cIdx = np.nonzero(edgeMap)
edgeCount = len(cIdx)

rCent = rIdx[None,:,None]-rAll[:,None,:]
cCent = cIdx[None,:,None]-cAll[:,None,:]

valid = (rCent>=0)&(rCent<r)&(cCent>=0)&(cCent<c)
flatIdx = (cCent*c+rCent)*radCount+np.arange(radCount)[:,None,None]
flatIdx = np.where(valid,flatIdx,0).ravel()

accumulator = np.bincount(flatIdx,minlength=r*c*radCount).reshape(r,c,radCount)




ValueError: cannot reshape array of size 3928515 into shape (412,512,15)